# excercise_8_4_LangGraph_RAG

**Temat:** RAG w grafie — index → retrieve → generate (szkic).

## Zadanie 1 — Index (chunk + embed)

In [ ]:
import numpy as np
def chunk_text(text: str, max_len: int = 100) -> list[str]:
    words, chunks, cur = text.split(), [], ""
    for w in words:
        if len(cur) + len(w) + 1 <= max_len:
            cur = (cur + " " + w).strip()
        else:
            chunks.append(cur); cur = w
    if cur: chunks.append(cur)
    return chunks

def embed(text: str) -> np.ndarray:
    words = text.split()
    return np.array([len(text), len(words), len(set(words))], dtype=float)  # TODO: prosta cecha

def indexer_node(state: dict) -> dict:
    chunks = chunk_text(state.get("docs",""), max_len=80)
    state["chunks"] = chunks
    state["vectors"] = [embed(c) for c in chunks]
    return state

s = indexer_node({"docs":"RAG łączy retrieval i generację. LangGraph modeluje workflowy jako grafy."})
assert "chunks" in s and "vectors" in s


## Zadanie 2 — Retriever (top-k cosine)

In [ ]:
def cosine(a: np.ndarray, b: np.ndarray) -> float:
    denom = (np.linalg.norm(a) * np.linalg.norm(b)) or 1e-12
    return float(np.dot(a, b) / denom)

def retriever_node(state: dict, top_k: int = 2) -> dict:
    qv = embed(state.get("question",""))
    pairs = list(zip(state.get("chunks", []), state.get("vectors", [])))
    scored = sorted(((c, cosine(v, qv)) for c, v in pairs), key=lambda x: x[1], reverse=True)[:top_k]
    state["context"] = [c for c, _ in scored]
    return state

r = retriever_node(s | {"question":"retrieval generacja"})
assert "context" in r and len(r["context"]) >= 1


## Zadanie 3 — Generator (szkic) + graf

In [ ]:
try:
    from langgraph.graph import StateGraph, START, END
except Exception as e:
    print("LangGraph nie jest zainstalowany (demo).", e)
    StateGraph = object; START="__START__"; END="__END__"

def generator_node(state: dict) -> dict:
    ctx = state.get("context", [])
    q = state.get("question","")
    state["answer"] = f"Kontekst={ctx[:1]} | Odpowiedź: {q}"  # TODO: sformatuj inaczej
    return state

builder = StateGraph(dict)
try:
    builder.add_node("index", indexer_node)
    builder.add_node("retrieve", retriever_node)
    builder.add_node("generate", generator_node)
    builder.add_edge(START, "index")
    builder.add_edge("index", "retrieve")
    builder.add_edge("retrieve", "generate")
    builder.add_edge("generate", END)
    # graph = builder.compile()  # nie uruchamiamy
    print("Szkic grafu RAG gotowy.")
except Exception as e:
    print("Szkic budowy grafu:", e)
